## Load Data
2024-04-22

This notebook loads all the precalculated RRI scores, fundamentals and any data stored in files.

In [1]:
import pandas as pd
import numpy as np
import csv
import ujson 
import json
from tqdm.notebook import tqdm
from pandas import Timestamp
from datetime import datetime, timedelta

In [ ]:
# ISIN to RR ID
rr_isin = pd.read_csv('data/REPRISK_ISINs.csv', quoting=csv.QUOTE_NONE)
rr_isin.columns = ['rr', 'ISIN']
rr_isin['ISIN'] = rr_isin['ISIN'].str.replace('\"', '')
map_isin = rr_isin.set_index('ISIN')['rr'].to_dict()

# Incident Data
incident_data = pd.read_csv('data/REPRISK_EVENTS.csv', quoting=csv.QUOTE_NONE)
incident_data.columns = ['date', 'ID_incident', 'ID_RR', 'reach', 'severity', 'unsharp', 'novelty', 
                         'environment', 'social', 'government', 'crosscutting']

incident_data['date'] = pd.to_datetime(incident_data['date'])
incident_data['severity'].replace({0: 1}, inplace=True)
incident_data['reach'].replace({0: 1}, inplace=True)
incident_data['novelty'].replace({0: 1}, inplace=True)

severity_weights = {1: 1, 
                    2: 10, 
                    3: 100}
reach_weights = {1: 1, 
                 2: 2, 
                 3: 3}
novelty_weights = {1: 1,
                   2: 2}

incident_data["Incident Score"] = (incident_data['severity'].replace(severity_weights)
                                   *incident_data['reach'].replace(reach_weights)
                                   *incident_data['novelty'].replace(novelty_weights))


In [ ]:
# read spx historical constituents
const = pd.read_csv("data/spx_constituents.csv", quoting=csv.QUOTE_NONE, delimiter=' ')
const.columns = ['date'] + [x.replace('\"', '') for x in const.columns[1:]]
const['date'] = const['date'].str.replace('\"', '')
const.set_index('date', inplace=True)

# map each date to its spx constituents
spx_const = {}

for i in tqdm(range(len(const))):
    row = const.iloc[i]
    date = const.index[i]
    spx_const[date] = set()
    for isin in const.columns:
        if row[isin] == 1:
            spx_const[date].add(isin)

# all the spx isin constituents ever
all_spx_isin = set(const.columns)

# read historical euro constituents list
const = pd.read_csv("data/stoxx_constituents.csv", quoting=csv.QUOTE_NONE, delimiter=' ')
const.columns = ['date'] + [x.replace('\"', '') for x in const.columns[1:]]
const['date'] = const['date'].str.replace('\"', '')
const.set_index('date', inplace=True)

# map each date to its eurostoxx constituents
euro_const = {}

for i in tqdm(range(len(const))):
    row = const.iloc[i]
    date = const.index[i]
    euro_const[date] = set()
    for isin in const.columns:
        if row[isin] == 1:
            euro_const[date].add(isin)

all_euro_isin = set(const.columns)

In [ ]:
def convert_strings_to_timestamps(d):
    for isin, nested_dict in tqdm(d.items()):
        d[isin] = {Timestamp(timestamp): inner_dict for timestamp, inner_dict in nested_dict.items()}
    return d

with open('data/spx_esg.json', 'r') as json_file:
    loaded_spx_esg = ujson.load(json_file)

spx_esg = convert_strings_to_timestamps(loaded_spx_esg)

with open('data/euro_esg.json', 'r') as json_file:
    loaded_euro_esg = ujson.load(json_file)

euro_esg = convert_strings_to_timestamps(loaded_euro_esg)

In [ ]:
# sector data for SPX
sector_data = pd.read_csv("data/SPsectors.csv", quoting=csv.QUOTE_NONE)
spx_sector = {sector_data['ISIN'].iloc[i]: sector_data['SP_sectors'].iloc[i] for i in range(len(sector_data))}

# sector data for Eurostoxx
euro_sector_data = pd.read_csv("data/stoxx_TR.TRBCEconomicSector.csv", quoting=csv.QUOTE_NONE)
euro_sector_data.columns = ['num', 'isin', 'sector']
for key in ['num', 'isin', 'sector']:
    euro_sector_data[key] = euro_sector_data[key].str.replace('\"', '')
euro_sector = {}
for i in range(len(euro_sector_data)):
    euro_sector[euro_sector_data['isin'].iloc[i]] = euro_sector_data['sector'].iloc[i]

In [ ]:
# SPX marketcaps
spx_mc = pd.read_csv("data/spx_TR.CompanyMarketCap.csv", quoting=csv.QUOTE_NONE)

spx_mc.columns = ['num', 'isin', 'market_cap', 'date']

for key in ['num', 'isin', 'market_cap', 'date']:
    spx_mc[key] = spx_mc[key].str.replace('\"', '')

spx_mc = spx_mc.replace(r'^\s*$', np.nan, regex=True)

spx_mc = spx_mc.dropna()

spx_mc['market_cap'] = pd.to_numeric(spx_mc['market_cap'])

spx_mc['date'] = spx_mc['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
spx_mc.set_index('isin', inplace=True)
spx_mc = spx_mc.drop('num', axis=1)

# Convert to a 2D dictionary (isin -> date -> row)
spx_mc['isin2'] = spx_mc.index
spx_mc = spx_mc.groupby('isin2').apply(
    lambda x: x.set_index('date').drop(columns='isin2').to_dict('index')
).to_dict()

In [ ]:
# Euro Marketcaps
euro_mc = pd.read_csv("data/stoxx_TR.CompanyMarketCap.csv", quoting=csv.QUOTE_NONE)

euro_mc.columns = ['num', 'isin', 'market_cap', 'date']

for key in ['num', 'isin', 'market_cap', 'date']:
    euro_mc[key] = euro_mc[key].str.replace('\"', '')

euro_mc = euro_mc.replace(r'^\s*$', np.nan, regex=True)

euro_mc = euro_mc.dropna()

euro_mc['market_cap'] = pd.to_numeric(euro_mc['market_cap'])

euro_mc['date'] = euro_mc['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
euro_mc.set_index('isin', inplace=True)
euro_mc = euro_mc.drop('num', axis=1)

# Convert to a 2D dictionary (isin -> date -> row)
euro_mc['isin2'] = euro_mc.index
euro_mc = euro_mc.groupby('isin2').apply(
    lambda x: x.set_index('date').drop(columns='isin2').to_dict('index')
).to_dict()

In [ ]:
# read SPX stock prices using fundamental data
spx_price = pd.read_csv("data/spx_TR.CLOSEPRICE.csv", quoting=csv.QUOTE_NONE)

spx_price.columns = ['num', 'isin', 'close', 'date']

for key in ['num', 'isin', 'close', 'date']:
    spx_price[key] = spx_price[key].str.replace('\"', '')
    
spx_price = spx_price.replace(r'^\s*$', np.nan, regex=True)
spx_price = spx_price.dropna()
spx_price['close'] =  pd.to_numeric(spx_price['close'])
spx_price['date'] = spx_price['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
spx_price.set_index('isin', inplace=True)
spx_price = spx_price.drop('num', axis=1)

spx_price = {isin: group for isin, group in tqdm(spx_price.groupby(spx_price.index))}

# read eurosotxx stock prices using fundamental data
euro_price = pd.read_csv("data/stoxx_TR.CLOSEPRICE.csv", quoting=csv.QUOTE_NONE)

euro_price.columns = ['num', 'isin', 'close', 'date']

for key in ['num', 'isin', 'close', 'date']:
    euro_price[key] = euro_price[key].str.replace('\"', '')
    
euro_price = euro_price.replace(r'^\s*$', np.nan, regex=True)
euro_price = euro_price.dropna()
euro_price['close'] =  pd.to_numeric(euro_price['close'])
euro_price['date'] = euro_price['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
euro_price.set_index('isin', inplace=True)
euro_price = euro_price.drop('num', axis=1)

euro_price = {isin: group for isin, group in tqdm(euro_price.groupby(euro_price.index))}

In [ ]:
# trend isin
spx_trend_esg = spx_esg.copy()

for isin in tqdm(spx_trend_esg):
    spx_trend_esg[isin] = pd.DataFrame(spx_trend_esg[isin]).T

for isin in tqdm(spx_trend_esg):
    spx_trend_esg[isin] = spx_trend_esg[isin].apply(lambda x: x - x.shift(30)).dropna()

for isin in tqdm(spx_trend_esg):
    spx_trend_esg[isin] = spx_trend_esg[isin].to_dict(orient='index')

In [ ]:
# trend isin
euro_trend_esg = euro_esg.copy()

for isin in tqdm(euro_trend_esg):
    euro_trend_esg[isin] = pd.DataFrame(euro_trend_esg[isin]).T

for isin in tqdm(euro_trend_esg):
    euro_trend_esg[isin] = euro_trend_esg[isin].apply(lambda x: x - x.shift(30)).dropna()

for isin in tqdm(euro_trend_esg):
    euro_trend_esg[isin] = euro_trend_esg[isin].to_dict(orient='index')

In [ ]:
ff = pd.read_csv('data/stoxx_TR.FreeFloatPct.csv', quoting=csv.QUOTE_NONE)
ff.columns = ['RR_id', 'ISIN', 'Free Float', 'date']

for key in ['RR_id', 'ISIN', 'Free Float', 'date']:
    ff[key] = ff[key].str.replace('\"', '')
ff['Free Float'].replace('', np.nan, inplace=True)
ff = ff.dropna()
ff = ff.reindex(index=ff.index[::-1])
ff['Free Float'] =  pd.to_numeric(ff['Free Float'])
ff['Free Float'] = ff['Free Float']/100

euro_freefloat = {}
for isin in tqdm(set(ff['ISIN'])):
    euro_freefloat[isin] = ff[ff['ISIN'] == isin]
    euro_freefloat[isin]['date'] = pd.to_datetime(euro_freefloat[isin]['date']).dt.strftime('%Y-%m-%d')

In [206]:
spx_data = {}
all_dates = list(set(weekly_dates_spx + quarterly_dates_spx + monthly_dates_spx))
all_dates.sort()
for date in tqdm(all_dates):
    if len(spx_data.get(date, [])):
        continue
    if len(spx_const.get(date, [])):
        spx_data[date] = {}
    for isin in spx_const.get(date, []):
        spx_data[date][isin] = {}
        try:
            spx_data[date][isin]['price'] = float(spx_price[isin][spx_price[isin]['date'] == date]['close'])
        except Exception:
            spx_data[date][isin]['price'] = np.nan
        try:
            spx_data[date][isin]['mc'] = float(spx_mc[isin][pd.to_datetime(date)]['market_cap'])
        except Exception:
            spx_data[date][isin]['mc'] = np.nan
        try:
            spx_data[date][isin]['freefloat'] = float(spx_freefloat[isin][spx_freefloat[isin]['date'] <= date].iloc[-1]['Free Float'])
        except Exception:
            spx_data[date][isin]['freefloat'] = np.nan
        try:
            spx_data[date][isin]['esg_score'] = spx_esg[isin][pd.to_datetime(date)]['esg_score']
            spx_data[date][isin]['environment'] = spx_esg[isin][pd.to_datetime(date)]['environment']
            spx_data[date][isin]['social'] = spx_esg[isin][pd.to_datetime(date)]['social']
            spx_data[date][isin]['government'] = spx_esg[isin][pd.to_datetime(date)]['government']
        except Exception:
            spx_data[date][isin]['esg_score'] = np.nan
            spx_data[date][isin]['environment'] = np.nan
            spx_data[date][isin]['social'] = np.nan
            spx_data[date][isin]['government'] = np.nan

def map_sector(isin):
    return 'financials/util' if spx_sector.get(isin, '') in ['financials', 'utilities'] else 'other'    

for date in tqdm(all_dates):
    for isin in spx_data[date]:
        spx_data[date][isin]['sector'] = map_sector(isin)

for date in tqdm(all_dates):
    for isin in spx_data[date]:
        for m in ['esg_score', 'environment', 'social', 'government']:
            spx_data[date][isin]['trend_esg'] = spx_trend_esg.get(isin, {}).get(pd.to_datetime(date), {m: np.nan})[m]

  0%|                                                                                          | 0/874 [00:00<?, ?it/s]C:\Users\anish\AppData\Local\Temp\ipykernel_48432\1897908601.py:12: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

100%|████████████████████████████████████████████████████████████████████████████████| 874/874 [27:37<00:00,  1.90s/it]


In [597]:
for date in tqdm(all_dates):
    for isin in spx_data[date]:
        for m in ['esg_score', 'environment', 'social', 'government']:
            spx_data[date][isin][f'trend_{m}'] = spx_trend_esg.get(isin, {}).get(pd.to_datetime(date), {m: np.nan})[m]

100%|████████████████████████████████████████████████████████████████████████████████| 874/874 [36:04<00:00,  2.48s/it]


In [598]:
spx_df = pd.DataFrame.from_dict({(date, isin): spx_data[date][isin]
                             for date in spx_data.keys()
                             for isin in spx_data[date].keys()},
                            orient='index')

spx_df.index.names = ['date', 'isin']
spx_df = spx_df.dropna()
print(spx_df)

                              price            mc  freefloat  esg_score  \
date       isin                                                           
2010-04-07 US0846707026   79.950000  1.973382e+11   0.915311  59.109932   
           US04621X1081   34.900000  3.982446e+09   0.994575   0.000000   
           US26441C2044   49.110000  2.145343e+10   0.997549  52.527405   
           US37959E1029   23.973357  4.472798e+09   0.989762   0.000000   
           US7237871071   59.580000  6.898665e+09   0.991520  20.150154   
...                             ...           ...        ...        ...   
2024-04-01 US2567461080  135.910000  2.962607e+10   0.997188  60.162579   
           US0028241000  112.090000  1.944968e+11   0.993941  51.043027   
           US92343V1044   42.280000  1.777566e+11   0.999515  45.835268   
           US02079K3059  155.490000  1.938935e+12   0.997172  82.510117   
           US30212P3038  133.220000  1.815635e+10   0.993776  42.556004   

                        

In [348]:
euro_data = {}
all_dates = list(set(weekly_dates_euro + quarterly_dates_euro + monthly_dates_euro))
all_dates.sort()
for date in tqdm(all_dates):
    if len(euro_data.get(date, [])):
        continue
    if len(euro_const.get(date, [])):
        euro_data[date] = {}
    for isin in euro_const.get(date, []):
        euro_data[date][isin] = {}
        try:
            euro_data[date][isin]['price'] = float(euro_price[isin][euro_price[isin]['date'] == date]['close'])
        except Exception:
            euro_data[date][isin]['price'] = np.nan
        try:
            euro_data[date][isin]['mc'] = float(euro_mc[isin][pd.to_datetime(date)]['market_cap'])
        except Exception:
            euro_data[date][isin]['mc'] = np.nan
        try:
            euro_data[date][isin]['freefloat'] = float(euro_freefloat[isin][euro_freefloat[isin]['date'] <= date].iloc[-1]['Free Float'])
        except Exception:
            euro_data[date][isin]['freefloat'] = np.nan
        try:
            euro_data[date][isin]['esg_score'] = euro_esg[isin][pd.to_datetime(date)]['esg_score']
            euro_data[date][isin]['environment'] = euro_esg[isin][pd.to_datetime(date)]['environment']
            euro_data[date][isin]['social'] = euro_esg[isin][pd.to_datetime(date)]['social']
            euro_data[date][isin]['government'] = euro_esg[isin][pd.to_datetime(date)]['government']
        except Exception:
            euro_data[date][isin]['esg_score'] = np.nan
            euro_data[date][isin]['environment'] = np.nan
            euro_data[date][isin]['social'] = np.nan
            euro_data[date][isin]['government'] = np.nan

            
def map_sector(isin):
    return 'financials/util' if euro_sector.get(isin, '') in ['Financials', 'Utilities'] else 'other' 
    
for date in tqdm(all_dates):
    for isin in euro_data[date]:
        euro_data[date][isin]['sector'] = map_sector(isin)

  0%|                                                                                          | 0/873 [00:00<?, ?it/s]C:\Users\anish\AppData\Local\Temp\ipykernel_48432\2517774683.py:14: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

100%|██████████████████████████████████████████████████████████████████████████████| 873/873 [00:00<00:00, 2682.17it/s]


In [633]:
for date in tqdm(all_dates):
    for isin in euro_data[date]:
        for m in ['esg_score', 'environment', 'social', 'government']:
            euro_data[date][isin][f'trend_{m}'] = euro_trend_esg.get(isin, {}).get(pd.to_datetime(date), {m: np.nan})[m]

100%|████████████████████████████████████████████████████████████████████████████████| 873/873 [22:20<00:00,  1.54s/it]


In [634]:
euro_df = pd.DataFrame.from_dict({(date, isin): euro_data[date][isin]
                             for date in euro_data.keys()
                             for isin in euro_data[date].keys()},
                            orient='index')

euro_df.index.names = ['date', 'isin']
euro_df = euro_df.dropna()
print(euro_df)

                              price            mc  freefloat  esg_score  \
date       isin                                                           
2010-04-07 DK0060336014   17.522631  4.753126e+09   0.892605   0.000000   
           FI0009002422   38.368229  3.067874e+09   0.596787   0.000000   
           FR0013506730  652.443027  9.001676e+09   0.894601   0.000000   
           DE0007235301   20.371771  1.532816e+09   0.726919  11.607824   
           JE00B3CX4509  777.500000  1.482965e+09   0.986473   0.000000   
...                             ...           ...        ...        ...   
2024-03-28 GB00BKFB1C65    2.580084  6.113634e+09   0.980947  27.138459   
           SE0001174970   18.914761  3.227497e+09   0.690272  19.661509   
           GB00BGXQNP29    6.466007  6.475996e+09   0.846652  22.414288   
           IT0005508921    4.200000  5.309827e+09   0.607431  28.481266   
           GB00B06QFB75    8.541775  3.206056e+09   0.943779   0.000000   

                        